In [1]:
#Data saved in csv/excel
import glob, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Set two decimal points for display
pd.options.display.float_format = "{:.2f}".format

# 1. Load the data

In [3]:
data=pd.read_csv('C:\\Users\\tsb7592\\OneDrive - Northwestern University\\Analysis Progress\\Basic_Desc_output.csv')

In [4]:
data['PIN']=data['PIN'].str.upper()

In [5]:
data.head()

,DYAD,PIN,DeviceID,Assessment Name,Inst,ItemID,Response,Score,Position,ResponseTime,DateCreated,InstStarted,InstEnded,Age_month,Age_group
0,68,TX011C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Title,1,NaN,1,NaN,2021-12-05 12:10:00,2021-12-05 12:09:44,2021-12-05 12:11:22,20,"(18, 21]"
1,68,TX011C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Title,1,NaN,1,NaN,2021-12-05 12:10:00,2021-12-05 12:09:44,2021-12-05 12:11:22,40,"(35, 43]"
2,68,TX011C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Instructions,1,NaN,2,NaN,2021-12-05 12:10:00,2021-12-05 12:09:44,2021-12-05 12:11:22,20,"(18, 21]"
3,68,TX011C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Instructions,1,NaN,2,NaN,2021-12-05 12:10:00,2021-12-05 12:09:44,2021-12-05 12:11:22,40,"(35, 43]"
4,68,TX011C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL22,2,2.0,3,NaN,2021-12-05 12:11:22,2021-12-05 12:09:44,2021-12-05 12:11:22,20,"(18, 21]"


# 2. Filter the inst to be IBQ

In [6]:
df= data[data.Inst=='Infant Behavior Questionnaire - Revised - Very Short Form']

In [7]:
df.Response =pd.to_numeric(df.Response)
df.Score =pd.to_numeric(df.Score)

In [8]:
df.describe()

,DYAD,Response,Score,Position,ResponseTime,Age_month
count,1560.00,1560.00,1431.00,1560.00,1480.00,1560.00
mean,54.33,4.64,4.77,20.00,13.77,268.38
std,31.00,2.09,1.89,11.26,50.58,178.85
min,2.00,1.00,1.00,1.00,0.51,5.00
25%,31.00,3.00,3.00,10.00,4.49,10.00
50%,51.00,5.00,5.00,20.00,6.47,357.50
75%,79.25,6.00,6.00,30.00,11.09,396.00
max,105.00,8.00,7.00,39.00,1071.29,493.00


# 3. Rescore all items to ensure appropriate response coding

In [9]:
import numpy as np
#The score should equal the response for all items EXCEPT SelfReg_ibqrvsh11, for which the score should be 8-response 
df['Score'] = np.where(df.ItemID=='SelfReg_ibqrvsh11',8-df.Response, df.Response)

# 4. Calculate the three composite scores

In [10]:
a= ['SelfReg_ibqrvsh1','SelfReg_ibqrvsh2',
    'SelfReg_ibqrvsh7','SelfReg_ibqrvsh8', 
    'SelfReg_ibqrvsh13','SelfReg_ibqrvsh14', 'SelfReg_ibqrvsh15',
    'SelfReg_ibqrvsh20', 'SelfReg_ibqrvsh21',
    'SelfReg_ibqrvsh26', 'SelfReg_ibqrvsh27' ,
    'SelfReg_ibqrvsh36', 'SelfReg_ibqrvsh37']

b= ['SelfReg_ibqrvsh3', 'SelfReg_ibqrvsh4',
    'SelfReg_ibqrvsh9', 'SelfReg_ibqrvsh10',
    'SelfReg_ibqrvsh16', 'SelfReg_ibqrvsh17',  
    'SelfReg_ibqrvsh22', 'SelfReg_ibqrvsh23',  
    'SelfReg_ibqrvsh28', 'SelfReg_ibqrvsh29', 
    'SelfReg_ibqrvsh32', 'SelfReg_ibqrvsh33']


c= [ 'SelfReg_ibqrvsh5', 'SelfReg_ibqrvsh6',
    'SelfReg_ibqrvsh11', 'SelfReg_ibqrvsh12', 
    'SelfReg_ibqrvsh18', 'SelfReg_ibqrvsh19',
    'SelfReg_ibqrvsh24', 'SelfReg_ibqrvsh25',
    'SelfReg_ibqrvsh30', 'SelfReg_ibqrvsh31',
      'SelfReg_ibqrvsh34', 'SelfReg_ibqrvsh35']

In [11]:
SelfReg_SurgIBQR= df[df.ItemID.isin(a)].groupby(['PIN','DYAD'])['Score'].mean()
SelfReg_NegAffIBQR= df[df.ItemID.isin(b)].groupby(['PIN','DYAD'])['Score'].mean()
SelfReg_EffContIBQR= df[df.ItemID.isin(c)].groupby(['PIN','DYAD'])['Score'].mean()


In [12]:
comp =[SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR]

In [13]:
cdf = pd.concat(comp,axis=1)
cdf.columns = ['SelfReg_SurgIBQR','SelfReg_NegAffIBQR','SelfReg_EffContIBQR']

In [14]:
cdf.corr()

,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
SelfReg_SurgIBQR,1.00,0.40,0.41
SelfReg_NegAffIBQR,0.40,1.00,0.04
SelfReg_EffContIBQR,0.41,0.04,1.00


In [15]:
cdf

,,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
PIN,DYAD,,,
MN001P,34,4.62,4.50,6.42
MN003P,36,4.23,3.17,6.42
MN004P,37,4.46,3.33,5.25
MN005P,38,3.92,4.83,5.83
MN008P,41,4.08,4.42,4.58
MN015P,48,4.62,4.58,5.42
MN021P,54,5.15,5.08,5.83
MO002P,86,6.62,4.50,7.08
MO008P,92,5.38,4.75,4.75


# 5. Calculate item-rest correlations for every item – e.g., what is the Pearson correlation for SelfReg_ibqrvsh1 with the average score for the 12 other items on SelfReg_SurgIQBR. There should be 37 item-rest correlations. 

In [16]:
items=['SelfReg_ibqrvsh1', 'SelfReg_ibqrvsh2', 'SelfReg_ibqrvsh3', 'SelfReg_ibqrvsh4',
       'SelfReg_ibqrvsh5', 'SelfReg_ibqrvsh6', 'SelfReg_ibqrvsh7',
       'SelfReg_ibqrvsh8', 'SelfReg_ibqrvsh9', 'SelfReg_ibqrvsh10',
       'SelfReg_ibqrvsh11', 'SelfReg_ibqrvsh12', 'SelfReg_ibqrvsh13',
       'SelfReg_ibqrvsh14', 'SelfReg_ibqrvsh15', 'SelfReg_ibqrvsh16',
       'SelfReg_ibqrvsh17', 'SelfReg_ibqrvsh18', 'SelfReg_ibqrvsh19',
       'SelfReg_ibqrvsh20', 'SelfReg_ibqrvsh21', 'SelfReg_ibqrvsh22',
       'SelfReg_ibqrvsh23', 'SelfReg_ibqrvsh24', 'SelfReg_ibqrvsh25',
       'SelfReg_ibqrvsh26', 'SelfReg_ibqrvsh27', 'SelfReg_ibqrvsh28',
       'SelfReg_ibqrvsh29', 'SelfReg_ibqrvsh30', 'SelfReg_ibqrvsh31',
       'SelfReg_ibqrvsh32', 'SelfReg_ibqrvsh33', 'SelfReg_ibqrvsh34',
       'SelfReg_ibqrvsh35', 'SelfReg_ibqrvsh36', 'SelfReg_ibqrvsh37']

In [17]:
from scipy.stats import pearsonr

def item_rest(itemlist, li=None):
    if not li:
        li ={}
        
    for i in range(len(itemlist)):
        rest = itemlist.copy()
        rest.pop(i)
        list1 = df[df.ItemID==itemlist[i]].groupby(['PIN','DYAD'])['Score'].mean()
        list2  = df[df.ItemID.isin(rest)].groupby(['PIN','DYAD'])['Score'].mean()
    
    # Apply the pearsonr()
        corr, _ = pearsonr(list1, list2)
        print(itemlist[i], ': Pearsons correlation is %.3f ' % corr)
        li[itemlist[i]] = corr
    return li

In [18]:
itemrest = item_rest(a)

SelfReg_ibqrvsh1 : Pearsons correlation is 0.237 
SelfReg_ibqrvsh2 : Pearsons correlation is 0.180 
SelfReg_ibqrvsh7 : Pearsons correlation is 0.446 
SelfReg_ibqrvsh8 : Pearsons correlation is 0.495 
SelfReg_ibqrvsh13 : Pearsons correlation is 0.196 
SelfReg_ibqrvsh14 : Pearsons correlation is 0.285 
SelfReg_ibqrvsh15 : Pearsons correlation is 0.155 
SelfReg_ibqrvsh20 : Pearsons correlation is 0.227 
SelfReg_ibqrvsh21 : Pearsons correlation is 0.242 
SelfReg_ibqrvsh26 : Pearsons correlation is 0.376 
SelfReg_ibqrvsh27 : Pearsons correlation is 0.360 
SelfReg_ibqrvsh36 : Pearsons correlation is 0.348 
SelfReg_ibqrvsh37 : Pearsons correlation is 0.197 


In [19]:
itemrest =item_rest(b, itemrest)

SelfReg_ibqrvsh3 : Pearsons correlation is 0.474 
SelfReg_ibqrvsh4 : Pearsons correlation is 0.603 
SelfReg_ibqrvsh9 : Pearsons correlation is 0.343 
SelfReg_ibqrvsh10 : Pearsons correlation is 0.265 
SelfReg_ibqrvsh16 : Pearsons correlation is 0.467 
SelfReg_ibqrvsh17 : Pearsons correlation is 0.326 
SelfReg_ibqrvsh22 : Pearsons correlation is -0.052 
SelfReg_ibqrvsh23 : Pearsons correlation is 0.548 
SelfReg_ibqrvsh28 : Pearsons correlation is 0.373 
SelfReg_ibqrvsh29 : Pearsons correlation is 0.409 
SelfReg_ibqrvsh32 : Pearsons correlation is 0.406 
SelfReg_ibqrvsh33 : Pearsons correlation is 0.448 


In [20]:
itemrest = item_rest(c,itemrest)

SelfReg_ibqrvsh5 : Pearsons correlation is 0.262 
SelfReg_ibqrvsh6 : Pearsons correlation is 0.312 
SelfReg_ibqrvsh11 : Pearsons correlation is 0.179 
SelfReg_ibqrvsh12 : Pearsons correlation is 0.495 
SelfReg_ibqrvsh18 : Pearsons correlation is 0.530 
SelfReg_ibqrvsh19 : Pearsons correlation is 0.445 
SelfReg_ibqrvsh24 : Pearsons correlation is 0.411 
SelfReg_ibqrvsh25 : Pearsons correlation is 0.532 
SelfReg_ibqrvsh30 : Pearsons correlation is 0.437 
SelfReg_ibqrvsh31 : Pearsons correlation is 0.466 
SelfReg_ibqrvsh34 : Pearsons correlation is 0.510 
SelfReg_ibqrvsh35 : Pearsons correlation is 0.456 


In [21]:
irc = pd.DataFrame.from_dict(itemrest, orient='index', columns=['item-rest-correlation'])
irc.round(3).to_csv('C:/Users/tsb7592/OneDrive - Northwestern University/Analysis Progress/IBQ_item_rest_correlation.csv')

# 6. Describe each of the 3 composite scores overall and by age group. NOTE: the Infant Behavior Questionnaire is not applicable to all age groups. 

In [22]:
cdf=cdf.reset_index()

In [23]:
spin = cdf.PIN.to_list()
cpin = []
for pin in spin:
    pin= pin[:5]+'C'
    cpin.append(pin)

In [24]:
cpin

['MN001C',
 'MN003C',
 'MN004C',
 'MN005C',
 'MN008C',
 'MN015C',
 'MN021C',
 'MO002C',
 'MO008C',
 'MO009C',
 'MO010C',
 'MO014C',
 'MO015C',
 'MO018C',
 'MO021C',
 'TX002C',
 'TX008C',
 'TX010C',
 'TX016C',
 'TX017C',
 'TX018C',
 'TX020C',
 'TX021C',
 'TX022C',
 'TX023C',
 'TX024C',
 'WI002C',
 'WI003C',
 'WI008C',
 'WI00PC',
 'WI012C',
 'WI015C',
 'WI016C',
 'WI019C',
 'WI021C',
 'WI028C',
 'WI032C']

In [25]:
child =data[data.PIN.isin(cpin)]

In [26]:
child[child.Age_group.notnull()]['PIN'].nunique()

34

In [27]:
cdf

,PIN,DYAD,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
0,MN001P,34,4.62,4.50,6.42
1,MN003P,36,4.23,3.17,6.42
2,MN004P,37,4.46,3.33,5.25
3,MN005P,38,3.92,4.83,5.83
4,MN008P,41,4.08,4.42,4.58
5,MN015P,48,4.62,4.58,5.42
6,MN021P,54,5.15,5.08,5.83
7,MO002P,86,6.62,4.50,7.08
8,MO008P,92,5.38,4.75,4.75
9,MO009P,93,5.69,4.75,4.75


In [28]:
de = pd.merge(cdf, child[['PIN','DYAD','Age_month','Age_group']], on=['PIN','DYAD'], how ='left')

In [29]:
de.groupby(['Age_group'])['SelfReg_SurgIBQR'].describe()

,count,mean,std,min,25%,50%,75%,max
Age_group,,,,,,,,
"(215, 1000]",103.00,4.00,0.00,4.00,4.00,4.00,4.00,4.00


In [30]:
de.groupby(['Age_group'])['SelfReg_NegAffIBQR'].describe()

,count,mean,std,min,25%,50%,75%,max
Age_group,,,,,,,,
"(215, 1000]",103.00,3.92,0.00,3.92,3.92,3.92,3.92,3.92


In [31]:
de.groupby(['Age_group'])['SelfReg_EffContIBQR'].describe()

,count,mean,std,min,25%,50%,75%,max
Age_group,,,,,,,,
"(215, 1000]",103.00,4.00,0.00,4.00,4.00,4.00,4.00,4.00


In [32]:
de =de.drop_duplicates()

In [33]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:/Users/tsb7592/OneDrive - Northwestern University/Analysis Progress/IBQ_composite_scores.xlsx', engine='xlsxwriter')

de.groupby(['Age_group'])['SelfReg_SurgIBQR'].describe().round(2).to_excel(writer, sheet_name='SurgIBQR')

de.groupby(['Age_group'])['SelfReg_NegAffIBQR'].describe().round(2).to_excel(writer, sheet_name='NegAffIBQR')

de.groupby(['Age_group'])['SelfReg_EffContIBQR'].describe().round(2).to_excel(writer, sheet_name='EffContIBQR')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [34]:
de['ChildPIN']  = de['PIN'].str[:-1] +'C'

In [35]:
de =de[(de.Age_month <42) | (de.Age_month.isnull())]
de

,PIN,DYAD,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR,Age_month,Age_group,ChildPIN
0,MN001P,34,4.62,4.50,6.42,NaN,NaN,MN001C
1,MN003P,36,4.23,3.17,6.42,NaN,NaN,MN003C
2,MN004P,37,4.46,3.33,5.25,NaN,NaN,MN004C
3,MN005P,38,3.92,4.83,5.83,NaN,NaN,MN005C
4,MN008P,41,4.08,4.42,4.58,NaN,NaN,MN008C
5,MN015P,48,4.62,4.58,5.42,NaN,NaN,MN015C
6,MN021P,54,5.15,5.08,5.83,NaN,NaN,MN021C
7,MO002P,86,6.62,4.50,7.08,NaN,NaN,MO002C
8,MO008P,92,5.38,4.75,4.75,NaN,NaN,MO008C
9,MO009P,93,5.69,4.75,4.75,NaN,NaN,MO009C


# 7. Add the other Self-Regulation instruments to the data file – that is, go back to the item-level data and select the Inst = “Playing with Bubbles” and “Toy Out of Reach”  

In [36]:
df2= data[(data.Inst=='Playing with Bubbles')]
df3 = data[(data.Inst=='Toy Out of Reach')]
df3.shape

(1474, 15)

# 8. Correlate the 3 composites with PA_2 and PA_3 from “Playing with Bubbles”  

In [37]:
df2['Score']=pd.to_numeric(df2['Score'])
df3['Score']=pd.to_numeric(df3['Score'])

In [38]:
pa2 =df2[df2.ItemID=='PA_2'][['PIN','DYAD','Score']]
pa3 =df2[df2.ItemID=='PA_3'][['PIN','DYAD','Score']]

PA = pd.merge(pa2,pa3, on=['PIN','DYAD'])
PA.columns =['PIN','DYAD','PA_2','PA_3']

In [39]:
PA

,PIN,DYAD,PA_2,PA_3
0,TX011C,68,2.00,2.00
1,TX011C,68,2.00,2.00
2,TX011C,68,2.00,2.00
3,TX011C,68,2.00,2.00
4,TX011C,69,3.00,3.00
...,...,...,...,...
106,WI025C,25,3.00,3.00
107,WI017C,17,2.00,2.00
108,MO022C,106,1.00,1.00
109,TX027C,84,3.00,3.00


In [40]:
cdf.DYAD=cdf.DYAD.to_string()

In [41]:
cdf.corrwith(PA['PA_2'])

SelfReg_SurgIBQR      0.28
SelfReg_NegAffIBQR    0.24
SelfReg_EffContIBQR   0.23
dtype: float64

In [42]:
c2 =cdf.corr().append(cdf.corrwith(PA['PA_2']), ignore_index=True)

In [43]:
c23 =c2.append(cdf.corrwith(PA['PA_3']), ignore_index=True)

In [44]:
c23

,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
0,1.00,0.40,0.41
1,0.40,1.00,0.04
2,0.41,0.04,1.00
3,0.28,0.24,0.23
4,0.30,0.24,0.25


# 9. Mean-level scores for 'toy out of reach'

In [45]:
df3.ItemID.unique()

array(['LabT_Title', 'SelfReg_LabTAB_Toy_Intro',
       'SelfReg_LabTAB_Toy_Baseline', 'SelfReg_Toy_Trial1Intro',
       'SelfReg_Toy_frus_reg1', 'SelfReg_Toy_frus_escalation1',
       'SelfReg_Toy_frus_cope1', 'SelfReg_Toy_Trial2Intro',
       'SelfReg_Toy_frus_reg2', 'SelfReg_Toy_frus_escalation2',
       'SelfReg _Toy_frus_cope2', 'SelfReg_Toy_Trial3Intro',
       'SelfReg_Toy_frus_reg3', 'SelfReg _Toy_frus_escalation3',
       'SelfReg_Toy _frus_cope3'], dtype=object)

In [46]:
#9a
f1 = df3[df3['ItemID'].str.contains("frus_reg")].groupby(['DYAD','PIN'])['Score'].mean().reset_index()

In [47]:
#9b
f2= df3[df3['ItemID'].str.contains("frus_escal")].groupby(['DYAD','PIN'])['Score'].mean().reset_index()

In [48]:
#9c
f3=df3[df3['ItemID'].str.contains("frus_cope")].groupby(['DYAD','PIN'])['Score'].mean().reset_index()
f3

,DYAD,PIN,Score
0,1,APPLETONC1,3.00
1,2,WI002C,3.00
2,3,WI003C,2.00
3,4,WI004,3.00
4,5,WI005C,2.00
...,...,...,...
88,104,MO020C,1.00
89,105,MO021C,1.00
90,106,MO022C,0.67
91,107,MO023C,1.00


In [49]:
f12 =pd.merge(f1,f2, on=['DYAD','PIN'])
frus =pd.merge(f12,f3, on=['DYAD','PIN'])

In [50]:
frus.columns =['DYAD','PIN', 'frus_reg', 'frus_escalation', 'frus_cope']

In [51]:
new =pd.merge(PA, frus,on=['DYAD','PIN'], how='left')

In [52]:
new.columns=['ChildPIN', 'DYAD','PA_2', 'PA_3', 'frus_reg', 'frus_escalation',
       'frus_cope']

In [53]:
new=new.drop_duplicates()
new.shape

(93, 7)

In [54]:
new

,ChildPIN,DYAD,PA_2,PA_3,frus_reg,frus_escalation,frus_cope
0,TX011C,68,2.00,2.00,3.00,0.00,3.00
4,TX011C,69,3.00,3.00,1.00,0.67,2.33
5,TX009C,66,0.00,1.00,3.00,0.00,3.00
6,TX005C,62,2.00,2.00,3.00,0.00,3.00
7,APPLETONC1,1,3.00,3.00,3.00,0.00,3.00
...,...,...,...,...,...,...,...
106,WI025C,25,3.00,3.00,0.67,0.00,3.00
107,WI017C,17,2.00,2.00,0.00,0.00,3.00
108,MO022C,106,1.00,1.00,1.33,1.33,0.67
109,TX027C,84,3.00,3.00,0.67,0.00,3.00


In [55]:
sc =pd.read_csv('C:/Users/tsb7592/OneDrive - Northwestern University/Analysis Progress/ScoreRecords_test2.csv')

In [56]:
sc=sc.drop(columns=[ 'SelfReg_SurgIBQR', 'SelfReg_NegAffIBQR',
       'SelfReg_EffContIBQR'])

#de has no age month data
de=de.drop(columns=[  'Age_month', 'Age_group'])

In [57]:
sc1= pd.merge(sc, de, on=['DYAD','ChildPIN'], how='left')

In [58]:
keep =['DYAD', 'ChildPIN', 'CaregiverPIN', 'Assessment Name', 'Age_month',
       'Age_group', 'SelfReg_SurgIBQR', 'SelfReg_NegAffIBQR',
       'SelfReg_EffContIBQR', 'SelfReg_SurgECBQ', 'SelfReg_NegAffECBQ',
       'SelfReg_EffContECBQ', 'SelfReg_SurgCBQ', 'SelfReg_NegAffCBQ',
       'SelfReg_EffContCBQ', 'PA_2', 'PA_3', 'frus_reg', 'frus_escalation',
       'frus_cope', 'CDI_Exp_theta', 'CDI_Rec_theta', 'Give_N', 'GlobalSR',
       'MYLLM', 'MemTest', 'Mul_Exp', 'Mul_Rec', 'PVT', 'NR_S', 'VerbCt',
       'ObjCt', 'PROMIS_CC_Int', 'PROMIS_PR', 'REMA', 'ShpM_Man', 'ShpM_I',
       'VDR', 'WHM', 'Phys_DP4_SS', 'AB_DP4_SS', 'SE_DP4_SS', 'Cog_DP4_SS',
       'Comm_DP4_SS', 'GDS_DP4_SS', 'Phys_DP4_GSV', 'AB_DP4_GSV', 'SE_DP4_GSV',
       'Cog_DP4_GSV', 'Comm_DP4_GSV', 'GDS_DP4_GSV', 'GM_MSEL_raw',
       'VR_MSEL_raw', 'FM_MSEL_raw', 'RL_MSEL_raw', 'EL_MSEL_raw',
       'GM_MSEL_Tscr', 'VR_MSEL_Tscr', 'FM_MSEL_Tscr', 'RL_MSEL_Tscr',
       'EL_MSEL_Tscr', 'ELC_MSEL_Tscr', 'GM_MSEL_AE', 'VR_MSEL_AE',
       'FM_MSEL_AE', 'RL_MSEL_AE', 'EL_MSEL_AE']

In [59]:
sc1[keep]

,DYAD,ChildPIN,CaregiverPIN,Assessment Name,Age_month,Age_group,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR,SelfReg_SurgECBQ,...,VR_MSEL_Tscr,FM_MSEL_Tscr,RL_MSEL_Tscr,EL_MSEL_Tscr,ELC_MSEL_Tscr,GM_MSEL_AE,VR_MSEL_AE,FM_MSEL_AE,RL_MSEL_AE,EL_MSEL_AE
0,1,WI001C,WI001P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,WI002C,WI002P,NaN,NaN,NaN,4.46,4.75,5.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,WI003C,WI003P,NaN,NaN,NaN,4.15,3.50,3.83,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,WI004C,WI004P,NaN,NaN,NaN,NaN,NaN,NaN,5.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,WI005C,WI005P,NaN,28.00,"(27, 30]",NaN,NaN,NaN,5.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,104,MO020C,MO020P,NaN,NaN,NaN,NaN,NaN,NaN,5.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,105,MO021C,MO021P,NaN,NaN,NaN,6.38,6.33,5.67,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,106,MO022C,MO022P,NaN,NaN,NaN,NaN,NaN,NaN,5.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,107,MO023C,MO023P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
sc2= pd.merge(sc1[keep], new, on=['DYAD','ChildPIN'], how='left')

In [61]:
sc2['PA_2']= sc2['PA_2_x'].fillna(sc2['PA_2_y'])
sc2['PA_3']= sc2['PA_3_x'].fillna(sc2['PA_3_y'])
sc2['frus_reg'] =sc2['frus_reg_x'].fillna(sc2['frus_reg_y'])
sc2['frus_escalation'] =sc2['frus_escalation_x'].fillna(sc2['frus_escalation_y'])
sc2['frus_cope'] =sc2['frus_cope_x'].fillna(sc2['frus_cope_y'])

In [62]:
sc2[keep].to_csv('C:/Users/tsb7592/OneDrive - Northwestern University/Analysis Progress/ScoreRecords_test3.csv', index=False)

In [63]:
cf =c23.append(cdf.corrwith(new['frus_reg']), ignore_index=True)
cf1 =cf.append(cdf.corrwith(new['frus_escalation']), ignore_index=True)
cf2 =cf1.append(cdf.corrwith(new['frus_cope']), ignore_index=True)
cf2

,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
0,1.00,0.40,0.41
1,0.40,1.00,0.04
2,0.41,0.04,1.00
3,0.28,0.24,0.23
4,0.30,0.24,0.25
5,-0.08,0.17,0.15
6,-0.42,-0.28,-0.58
7,0.38,0.20,0.43


In [64]:
cf2.index=['SelfReg_SurgIBQR', 'SelfReg_NegAffIBQR', 'SelfReg_EffContIBQR','PA_2','PA_3','frus_reg', 'frus_escalation', 'frus_cope']

In [65]:
cf2

,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
SelfReg_SurgIBQR,1.00,0.40,0.41
SelfReg_NegAffIBQR,0.40,1.00,0.04
SelfReg_EffContIBQR,0.41,0.04,1.00
PA_2,0.28,0.24,0.23
PA_3,0.30,0.24,0.25
frus_reg,-0.08,0.17,0.15
frus_escalation,-0.42,-0.28,-0.58
frus_cope,0.38,0.20,0.43


In [66]:
cf2.round(2).to_csv('C:/Users/tsb7592/OneDrive - Northwestern University/Analysis Progress/IBQ_corr.csv')